<a href="https://colab.research.google.com/github/RexPersicus/ChatGPT_Prompt_Eng_01/blob/main/Claude_RAG_LangGraph_multi_agent_email_gen_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#================================================================================================================
# This code uploads RAG files from a folder called uploads and gets a topic from the user
# and creates multi agents using langgraph to create an email for a target audience using the RAG files for better calibration.
#
# It is expecting the .env file
# RAG files can be .txt or .pdf only
#===================================================================================================================

In [1]:
# Install required packages
!pip install langchain langchain-community langgraph chromadb pypdf python-dotenv openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.2 MB/s eta 0:00:00

In [2]:
import os
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import Graph, StateGraph
from langchain_core.messages import HumanMessage, AIMessage
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv
import json
from typing import TypedDict, List

In [3]:
# Create uploads directory if it doesn't exist
#!mkdir -p uploads

# Load environment variables
load_dotenv()

# Get API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [4]:
# Define state schema
class AgentState(TypedDict):
    messages: List[str]
    topic: str
    recipient_type: str
    research_findings: str
    email_strategy: str
    final_email: str
    current_agent: str
    next_agent: str
    done: bool

def load_documents(folder_path='uploads'):
    """Load documents from a folder containing PDF and text files."""
    documents = []

    if not os.path.exists(folder_path):
        raise ValueError(f"Folder '{folder_path}' not found!")

    files = os.listdir(folder_path)
    if not files:
        raise ValueError(f"No files found in '{folder_path}' directory!")

    print(f"Found {len(files)} files in uploads folder:")
    for file in files:
        file_path = os.path.join(folder_path, file)
        print(f"Processing: {file}")
        if file.lower().endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
        elif file.lower().endswith('.txt'):
            loader = TextLoader(file_path)
            documents.extend(loader.load())
        else:
            print(f"Skipping {file} - not a PDF or TXT file")

    return documents

def create_vector_store(documents):
    """Create a vector store from the documents."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    texts = text_splitter.split_documents(documents)

    print(f"Created {len(texts)} text chunks for processing")
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(texts, embeddings)

    return vectorstore

# Initialize agents
llm = ChatOpenAI(temperature=0.7)



<ipython-input-4-08f60fac36d1>:54: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7)


In [16]:
class ResearchAgent:
    def __init__(self, vectorstore):
        self.retriever = vectorstore.as_retriever()
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True
        )

    def __call__(self, state: AgentState) -> dict:
        print("Research Agent: Analyzing documents...")

        query = f"""
        Analyze the provided documents and extract key information about:
        1. Our company's services/products
        2. Key selling points
        3. Relevant case studies or success stories
        4. Any specific information related to {state['topic']}

        Format the findings in a clear, structured way.
        """

        response = self.qa_chain.invoke({"query": query})
        state['research_findings'] = response['result']
        state['current_agent'] = 'research'
        state['next_agent'] = 'strategy'
        return state

In [17]:
class StrategyAgent:
    def __init__(self):
        self.llm = llm

    def __call__(self, state: AgentState) -> dict:
        print("Strategy Agent: Planning email approach...")

        prompt = f"""
        Based on the research findings below, develop a strategy for the marketing email:

        Research Findings:
        {state['research_findings']}

        Target Recipient: {state['recipient_type']}
        Topic Focus: {state['topic']}

        Create a strategic plan that includes:
        1. Key messages to emphasize
        2. Recommended tone and approach
        3. Specific points from the research to include
        4. Suggested structure for maximum impact
        """

        messages = [HumanMessage(content=prompt)]
        response = self.llm.invoke(messages)

        state['email_strategy'] = response.content
        state['current_agent'] = 'strategy'
        state['next_agent'] = 'writer'
        return state

In [18]:
class WriterAgent:
    def __init__(self):
        self.llm = llm

    def __call__(self, state: AgentState) -> dict:
        print("Writer Agent: Composing email...")

        prompt = f"""
        Create a compelling marketing email using the research and strategy below:

        Research Findings:
        {state['research_findings']}

        Email Strategy:
        {state['email_strategy']}

        Topic Focus: {state['topic']}
        Target Recipient: {state['recipient_type']}

        Generate a professional email that includes:
        1. Attention-grabbing subject line
        2. Personalized greeting
        3. Compelling body content
        4. Clear call to action
        5. Professional signature

        Make sure to incorporate specific details from the research and follow the recommended strategy.
        """

        messages = [HumanMessage(content=prompt)]
        response = self.llm.invoke(messages)

        state['final_email'] = response.content
        state['current_agent'] = 'writer'
        state['done'] = True
        return state

In [19]:
def should_continue(state: AgentState) -> str:
    """Determine if the workflow should continue or end."""
    if state['done']:
        return "end"
    return state['next_agent']

In [21]:
def create_agent_graph(vectorstore):
    # Initialize agents
    research_agent = ResearchAgent(vectorstore)
    strategy_agent = StrategyAgent()
    writer_agent = WriterAgent()

    # Create workflow graph
    workflow = StateGraph(AgentState)

    # Add agent nodes
    workflow.add_node('research', research_agent)
    workflow.add_node('strategy', strategy_agent)
    workflow.add_node('writer', writer_agent)

    # Define edges
    workflow.add_edge('research', 'strategy')
    workflow.add_edge('strategy', 'writer')

    # Set entry point
    workflow.set_entry_point('research')

    # Add conditional edge for completion
    workflow.add_conditional_edges(
        'writer',
        should_continue
    )

    # Set end point
    workflow.add_node("end", lambda x: x)

    # Compile workflow
    return workflow.compile()

In [22]:
def main():
    print("Welcome to the Multi-Agent Email Generator!")
    print("Please ensure your files are uploaded to the 'uploads' folder.")

    try:
        # Load documents and create vector store
        print("\nLoading documents...")
        documents = load_documents()
        vectorstore = create_vector_store(documents)

        # Create agent workflow
        workflow = create_agent_graph(vectorstore)

        while True:
            # Get user input
            print("\nWhat would you like to focus on in the email?")
            topic = input("Enter topic (or 'quit' to exit): ")

            if topic.lower() == 'quit':
                break

            recipient_type = input("Who is the target recipient? ")

            # Initialize state
            state = AgentState(
                messages=[],
                topic=topic,
                recipient_type=recipient_type,
                research_findings="",
                email_strategy="",
                final_email="",
                current_agent="",
                next_agent="research",
                done=False
            )

            # Run workflow
            print("\nGenerating email using multi-agent system...")
            final_state = workflow.invoke(state)

            # Display results
            print("\nGenerated Email:")
            print("-" * 50)
            print(final_state['final_email'])
            print("-" * 50)

            continue_gen = input("\nWould you like to generate another email? (yes/no): ")
            if continue_gen.lower() != 'yes':
                break

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Welcome to the Multi-Agent Email Generator!
Please ensure your files are uploaded to the 'uploads' folder.

Loading documents...
Found 2 files in uploads folder:
Processing: bell_canada.txt
Processing: royal_persicus.pdf
Created 4 text chunks for processing

What would you like to focus on in the email?
Enter topic (or 'quit' to exit): Write a great compelling email to market the services of the Royal Persicus company to Bell Canada.
Who is the target recipient? Bell CEO

Generating email using multi-agent system...
Research Agent: Analyzing documents...
Strategy Agent: Planning email approach...
Writer Agent: Composing email...

Generated Email:
--------------------------------------------------
Subject: Elevate Your Digital Transformation with Royal Persicus

Dear Bell Canada CEO,

I hope this message finds you well. At Royal Persicus, we understand the challenges faced by companies striving to excel in the competitive tech industry, just like Bell Canada.

Our specialized services i